# Incorporating Michel Veto Data

In [1]:
import sys
import os
import h5py
from collections import Counter
from progressbar import *
import re
import numpy as np
import h5py
from scipy import signal
import matplotlib
from repeating_classifier_training_utils import *
from functools import reduce

# Add the path to the parent directory to augment search for module
par_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

if par_dir not in sys.path:
    sys.path.append(par_dir)
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [2]:
veto_path = '/fast_scratch/WatChMaL/data/michels.h5'

In [4]:
mv_file = h5py.File(veto_path,'r')
mv_info = {}
for key in mv_file.keys():
    mv_info[key] = np.array(mv_file[key])

In [8]:
mv_info.keys()

dict_keys(['event_ids', 'is_michel', 'root_files'])

In [7]:
mv_info['is_michel'].shape

(3000000,)

## Load test set

In [9]:
# Get original h5 file info
# Import test events from h5 file
filtered_index = "/fast_scratch/WatChMaL/data/IWCD_fulltank_300_pe_idxs.npz"
filtered_indices = np.load(filtered_index, allow_pickle=True)
test_filtered_indices = filtered_indices['test_idxs']

original_data_path = "/data/WatChMaL/data/IWCDmPMT_4pi_fulltank_9M.h5"
f = h5py.File(original_data_path, "r")

original_eventids = np.array(f['event_ids'])
original_rootfiles = np.array(f['root_files'])

filtered_eventids = original_eventids[test_filtered_indices]
filtered_rootfiles = original_rootfiles[test_filtered_indices]

In [14]:
filtered_eventids.shape

(3351020,)

In [28]:
mv_dict = {}
pbar = ProgressBar(widgets=['Creating All-False Dictionary: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(filtered_eventids))
pbar.start()
for i in range(len(filtered_eventids)):
    mv_dict[(filtered_rootfiles[i], filtered_eventids[i])] = False
    pbar.update(i)
pbar.finish()

Creating All-False Dictionary: 100% [000000000000000000000000000] Time: 0:00:08


In [29]:
pbar = ProgressBar(widgets=['Update Muon Values Dictionary: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(mv_info['event_ids']))
pbar.start()
for i, is_michel in enumerate(mv_info['is_michel']):
    mv_dict[(mv_info['root_files'][i], mv_info['event_ids'][i])] = is_michel
    pbar.update(i)
pbar.finish()

Update Muon Values Dictionary: 100% [000000000000000000000000000] Time: 0:00:07


In [31]:
test_set_mv = []
pbar = ProgressBar(widgets=['Creating Veto Array: ', Percentage(), ' ', Bar(marker='0',left='[',right=']'),
           ' ', ETA()], maxval=len(filtered_rootfiles))
pbar.start()
for i in range(len(filtered_eventids)):
    test_set_mv.append(mv_dict[(filtered_rootfiles[i], filtered_eventids[i])])
    pbar.update(i)
pbar.finish()

Creating Veto Array: 100% [0000000000000000000000000000000000000] Time: 0:00:08


In [34]:
len(test_set_mv)

3351020

In [35]:
np.savez(os.path.join(os.getcwd(), 'Index_Storage/test_set_michel_veto.npz'), michel_veto=test_set_mv)